<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU45_Planificación y promoción del destino en base a los patrones en origen de los turistas</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > II - Data Processing Domain </font>
***
> # <font color='steelblue'> <b>05.- Data Collection</font>

<font color='steelblue'>Data Collection is the process to obtain and generate (if required) necessary data to model the problem.</font>

### 04. Transformar datos de movimiento de turistas nacionales por provincia

* Generar csv consolidado de datos de número de turistas por provincia de origen

Los datos se descargan de aquí, copiados a Input:

- Descarga directa de https://ine.es/dynt3/inebase/es/index.htm?padre=8578&capsel=8579
- Archivo: 53001.csv

Son datos de turistas nacionales por provincia de origen y municipio de destino a partir de los datos de antenas móviles publicados en INE

Se encuentran en INEBASE pero la descarga disrecta completa no es posible por la restricción de volumen. Se puede a futuro automatizar









In [1]:
## 53001: Número de turistas mensuales por CCAA y provincia de origen desagregados por municipio de destino
# t <- 53001
# groups_id <- get_tables(t, resource = "group")
# groups_id
# s <- get_tables(t, resource = "data")
# > s
# $status
# [1] "No puede mostrarse por restricciones de volumen"

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Settings" data-toc-modified-id="Settings-1"><font color="green">Settings</font></a></span></li><li><span><a href="#Data-Load" data-toc-modified-id="Data-Load-2"><font color="green">Data Load</font></a></span></li><li><span><a href="#ETL-Processes" data-toc-modified-id="ETL-Processes-3">ETL Processes</a></span><ul class="toc-item"><li><span><a href="#Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files" data-toc-modified-id="Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files-3.1">Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files</a></span></li></ul></li><li><span><a href="#Synthetic-Data-Generation" data-toc-modified-id="Synthetic-Data-Generation-4">Synthetic Data Generation</a></span></li><li><span><a href="#Fake-Data-Generation" data-toc-modified-id="Fake-Data-Generation-5">Fake Data Generation</a></span></li><li><span><a href="#Open-Data" data-toc-modified-id="Open-Data-6">Open Data</a></span></li><li><span><a href="#Data-Save" data-toc-modified-id="Data-Save-7"><font color="green">Data Save</font></a></span></li><li><span><a href="#Main-Conclusions" data-toc-modified-id="Main-Conclusions-8"><font color=" #2874a6 ">Main Conclusions</font></a></span></li><li><span><a href="#Main-Actions" data-toc-modified-id="Main-Actions-9"><font color=" #2874a6 ">Main Actions</font></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Acciones-done" data-toc-modified-id="Acciones-done-9.0.1"><font color=" #2874a6 ">Acciones done</font></a></span></li><li><span><a href="#Acctions-to-perform" data-toc-modified-id="Acctions-to-perform-9.0.2"><font color=" #2874a6 ">Acctions to perform</font></a></span></li></ul></li></ul></li></ul></div>

## <font color='green'>Settings</font>

### Encoding

Con la siguiente expresión se evitan problemas con el encoding al ejecutar el notebook. Es posible que deba ser eliminada o adaptada a la máquina en la que se ejecute el código.

In [2]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "es_ES.UTF-8/es_ES.UTF-8/es_ES.UTF-8/C/es_ES.UTF-8/C"

### Packages to use


- {tcltk} para selección interactiva de archivos locales
- {mapSpain} para obtener datos de municipios
- {readr} para leer y escribir archivos csv
- {dplyr} para explorar datos


In [3]:
library(readr)
library(mapSpain)
library(dplyr)
library(stringr)





Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### Paths

In [4]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

If there are more than one input file, make as many sections as files to import.

Instrucciones
- Los ficheros de entrada del proceso están siempre en Data/Input/.   
- Si hay más de un fichero de entrada, se crean tantos objetos iFile_xx y file_data_xx como ficheros de entrada (xx número correlativo con dos dígitos, rellenar con ceros a la izquierda)

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [5]:
# file_data <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [6]:
iFile <- "53001.csv"
file_data <- paste0(iPath, iFile)

if(file.exists(file_data)){
    cat("Se leerán datos del archivo: ", file_data)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/53001.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [7]:
data_01 <- read_delim(file_data, delim = ";", 
                     skip = 1,
                     escape_double = FALSE, trim_ws = TRUE,
                     col_types = "cccc-cc",
                     col_names = c("total_nacional", "total_ccaa", "provincia", "municipio_destino",
                        "periodo", "turistas"))


Estructura de  los datos:

In [8]:
data_01 |> glimpse()

Rows: 23,420,160
Columns: 6
$ total_nacional    <chr> "Total Nacional", "Total Nacional", "Total Nacional"…
$ total_ccaa        <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ provincia         <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ municipio_destino <chr> "Ababuj", "Ababuj", "Ababuj", "Ababuj", "Ababuj", "A…
$ periodo           <chr> "2022M10", "2022M09", "2022M08", "2022M07", "2022M06…
$ turistas          <chr> "46", "74", "237", "118", "148", "86", "192", "48", …


Muestra de datos:

In [9]:
data_01 |> slice_head(n = 5)

total_nacional,total_ccaa,provincia,municipio_destino,periodo,turistas
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Total Nacional,NA,NA,Ababuj,2022M10,46
Total Nacional,NA,NA,Ababuj,2022M09,74
Total Nacional,NA,NA,Ababuj,2022M08,237
Total Nacional,NA,NA,Ababuj,2022M07,118
Total Nacional,NA,NA,Ababuj,2022M06,148


## Open Data

Datos de municipios de {mapSpain} para obtener los códigos

In [10]:
data_02 <- esp_get_munic()

In [11]:
data_02 |> glimpse()

Rows: 8,131
Columns: 8
$ codauto       <chr> "01", "01", "01", "01", "01", "01", "01", "01", "01", "0…
$ ine.ccaa.name <chr> "Andalucía", "Andalucía", "Andalucía", "Andalucía", "And…
$ cpro          <chr> "04", "04", "04", "04", "04", "04", "04", "04", "04", "0…
$ ine.prov.name <chr> "Almería", "Almería", "Almería", "Almería", "Almería", "…
$ cmun          <chr> "001", "002", "003", "004", "005", "006", "007", "008", …
$ name          <chr> "Abla", "Abrucena", "Adra", "Albánchez", "Alboloduy", "A…
$ LAU_CODE      <chr> "04001", "04002", "04003", "04004", "04005", "04006", "0…
$ geometry      <GEOMETRY [°]> POLYGON ((-2.77744 37.23836..., POLYGON ((-2.88…


In [12]:
data_02 |> tibble() |> filter(cpro == "01")

codauto,ine.ccaa.name,cpro,ine.prov.name,cmun,name,LAU_CODE,geometry
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<GEOMETRY [°]>
16,País Vasco,01,Araba/Álava,001,Alegría-Dulantzi,01001,MULTIPOLYGON (((-2.52635 42...
16,País Vasco,01,Araba/Álava,002,Amurrio,01002,MULTIPOLYGON (((-3.01858 43...
16,País Vasco,01,Araba/Álava,003,Aramaio,01003,POLYGON ((-2.54726 43.08767...
16,País Vasco,01,Araba/Álava,004,Artziniega,01004,POLYGON ((-3.13397 43.09718...
16,País Vasco,01,Araba/Álava,006,Armiñón,01006,MULTIPOLYGON (((-2.88106 42...
16,País Vasco,01,Araba/Álava,008,Arratzua-Ubarrundia,01008,MULTIPOLYGON (((-2.55119 42...
16,País Vasco,01,Araba/Álava,009,Asparrena,01009,MULTIPOLYGON (((-2.33746 42...
16,País Vasco,01,Araba/Álava,010,Ayala / Aiara,01010,POLYGON ((-3.10886 43.14327...
16,País Vasco,01,Araba/Álava,011,Baños de Ebro / Mañueta,01011,POLYGON ((-2.69913 42.52747...


## ETL Processes

### Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files

Se han importado en el apartado Data Load anterior:

* Turismo interno por municipio en la comunidad de Madrid

Incluir apartados si procede para: Extracción de datos (select, filter), Transformación de datos, (mutate, joins, ...). Si es necesario tratar datos perdidos, indicarlo también en NB 09.2

Si no aplica: Estos datos no requieren tareas de este tipo.

#### Data transformation

* Obtener municipios de la Comunidad de Madrid

In [15]:
edata_02 <- data_02 |>
    tibble() |>
    filter(cpro == "28") |>
    select(cmun, name) 



* Unir con municipios coincidentes

In [16]:
edata_01 <- data_01 |> 
  inner_join(edata_02,
            by = c("municipio_destino" = "name")) 

In [49]:
edata_01 |> glimpse()

Rows: 521,280
Columns: 7
$ total_nacional    <chr> "Total Nacional", "Total Nacional", "Total Nacional"…
$ total_ccaa        <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ provincia         <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ municipio_destino <chr> "Acebeda, La", "Acebeda, La", "Acebeda, La", "Acebed…
$ periodo           <chr> "2022M10", "2022M09", "2022M08", "2022M07", "2022M06…
$ turistas          <chr> NA, NA, NA, "39", "125", "31", NA, NA, NA, ".", NA, …
$ cmun              <chr> "001", "001", "001", "001", "001", "001", "001", "00…


In [22]:
edata_01 |> filter(municipio_destino == "Ajalvir", periodo == "2022M10")

total_nacional,total_ccaa,provincia,municipio_destino,periodo,turistas,cmun
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Total Nacional,NA,NA,Ajalvir,2022M10,1.104,002
Total Nacional,Andalucía,NA,Ajalvir,2022M10,202,002
Total Nacional,Andalucía,Almería,Ajalvir,2022M10,NA,002
Total Nacional,Andalucía,Cádiz,Ajalvir,2022M10,NA,002
Total Nacional,Andalucía,Córdoba,Ajalvir,2022M10,NA,002
Total Nacional,Andalucía,Granada,Ajalvir,2022M10,NA,002
Total Nacional,Andalucía,Huelva,Ajalvir,2022M10,NA,002
Total Nacional,Andalucía,Jaén,Ajalvir,2022M10,NA,002
Total Nacional,Andalucía,Málaga,Ajalvir,2022M10,46,002


In [17]:
edata_01 |> slice_head(n = 5)

total_nacional,total_ccaa,provincia,municipio_destino,periodo,turistas,cmun
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Total Nacional,NA,NA,"Acebeda, La",2022M10,NA,001
Total Nacional,NA,NA,"Acebeda, La",2022M09,NA,001
Total Nacional,NA,NA,"Acebeda, La",2022M08,NA,001
Total Nacional,NA,NA,"Acebeda, La",2022M07,39,001
Total Nacional,NA,NA,"Acebeda, La",2022M06,125,001


* Transformar periodo a mes para poder unir después, identificar NAs por secreto estadístico y convertir a numérica el número de turistas

In [32]:
tdata <- edata_01 |>
    mutate(mes = str_replace(periodo, "M", "-")) |>
    select(-periodo) |>
    mutate(secreto = if_else(turistas == ".", 1, 0)) |>
    mutate(turistas = str_replace_all(turistas, "\\.", "")) |>
    mutate(turistas = as.numeric(turistas))

In [33]:
tdata |> glimpse()

Rows: 521,280
Columns: 8
$ total_nacional    <chr> "Total Nacional", "Total Nacional", "Total Nacional"…
$ total_ccaa        <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ provincia         <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ municipio_destino <chr> "Acebeda, La", "Acebeda, La", "Acebeda, La", "Acebed…
$ turistas          <dbl> NA, NA, NA, 39, 125, 31, NA, NA, NA, NA, NA, NA, NA,…
$ cmun              <chr> "001", "001", "001", "001", "001", "001", "001", "00…
$ mes               <chr> "2022-10", "2022-09", "2022-08", "2022-07", "2022-06…
$ secreto           <dbl> NA, NA, NA, 0, 0, 0, NA, NA, NA, 1, NA, NA, NA, NA, …


In [34]:
tdata |> filter(municipio_destino == "Ajalvir", mes == "2022-10")

total_nacional,total_ccaa,provincia,municipio_destino,turistas,cmun,mes,secreto
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Total Nacional,NA,NA,Ajalvir,1104,002,2022-10,0
Total Nacional,Andalucía,NA,Ajalvir,202,002,2022-10,0
Total Nacional,Andalucía,Almería,Ajalvir,NA,002,2022-10,NA
Total Nacional,Andalucía,Cádiz,Ajalvir,NA,002,2022-10,NA
Total Nacional,Andalucía,Córdoba,Ajalvir,NA,002,2022-10,NA
Total Nacional,Andalucía,Granada,Ajalvir,NA,002,2022-10,NA
Total Nacional,Andalucía,Huelva,Ajalvir,NA,002,2022-10,NA
Total Nacional,Andalucía,Jaén,Ajalvir,NA,002,2022-10,NA
Total Nacional,Andalucía,Málaga,Ajalvir,46,002,2022-10,0


In [54]:
tdata |> slice_head(n = 5)

total_nacional,total_ccaa,provincia,municipio_destino,turistas,cmun,mes,secreto
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Total Nacional,NA,NA,"Acebeda, La",NA,001,2022-10,NA
Total Nacional,NA,NA,"Acebeda, La",NA,001,2022-09,NA
Total Nacional,NA,NA,"Acebeda, La",NA,001,2022-08,NA
Total Nacional,NA,NA,"Acebeda, La",39,001,2022-07,0
Total Nacional,NA,NA,"Acebeda, La",125,001,2022-06,0


## Synthetic Data Generation

No aplica

## Fake Data Generation

No aplica

## <font color='green'>Data Save</font>

Este proceso, puede copiarse y repetirse en aquellas partes del notebbok que necesiten guardar datos.
Recuerde cambiar las cadenas añadida del fichero para diferenciarlas

<font color='tomato'> Identificamos los datos a guardar</font>

In [36]:
data_to_save <- tdata


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_05".
* Número de la tarea que lo genera, por ejemplo "_01"
* En caso de generarse varios ficheros en la misma tarea, llevarán _01 _02 ... después
* Nombre: identificativo de "properData", por ejemplo "_zonasgeo"
* Extensión del archivo

Ejemplo: "CU_04_05_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas)

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 05

In [37]:
caso <- "CU_45"
proceso <- '_05'
tarea <- "_04"
archivo <- ""
proper <- "_interno_prov"
extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [38]:
# file_save <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save, path_out)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [39]:
file_save <- paste0(caso, proceso, tarea, archivo, proper, extension) 
path_out <- paste0(oPath, file_save)
write_csv(data_to_save, path_out)

cat('File saved as: ')
path_out

File saved as: 

[1] "Data/Output/CU_45_05_04_interno_prov.csv"

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [40]:
path_in <- paste0(iPath, file_save)
file.copy(path_out, path_in, overwrite = TRUE)

[1] TRUE

## <font color=' #2874a6 '>Main Conclusions</font>

<font color='steelblue'>List and describe the general conclusions of the analysis carried out​.</font>

### Prerequisites

Para que funcione este código se necesita:

* Las rutas de archivos `Data/Input` y `Data/Output` deben existir (relativas a la ruta del _notebook_)
* El paquete tcltk instalado para seleccionar archivos interactivamente. No se necesita en producción.
* Los paquetes readr, dplyr, readxl, stringr deben estar instalados.


### Configuration Management

This notebook has been tested with the following versions of R and packages. It cannot be assured that later versions work in the same way:
* R 4.2.2
* tcltk 4.2.2
* readr 2.1.3
* dplyr 1.1.0
* readxl 1.4.1
* stringr 1.5.0

### Data structures

#### Objeto `data`

- Hay 521280 filas con información de las siguientes variables:
	* total_nacional
	* total_ccaa
	* provincia
	* municipio_destino
	* periodo
	* turistas
	* cmun


#### Observaciones generales sobre los datos

* Además de los valores de provincias, vienen datos totales por comunidad autónoma y total país
* En teoría serían la suma de las provincias que componen las regiones, pero podría no coincidir por el secreto estadístico (pocos datos en ununa provincia determinada, que sí se suman al total pero no se publican)
* Vienen dos tipos de valores perdidos: NA y un punto, que es valor no publicado
* Los NA es posible que sean cero, pero al no estar seguros se dejan como NA
* Los datos disponibles en el archivo utilizado son en este rango de meses

In [41]:
data_to_save |> pull(mes) |> range()

[1] "2019-07" "2022-10"

### Consideraciones para despliegue en piloto

* No aplica

### Consideraciones para despliegue en producción

* Se deben crear los procesos ETL en producción necesarios para que los datos de entrada estén actualizados

## <font color=' #2874a6 '>Main Actions</font>

#### <font color=' #2874a6 '>Acciones done</font>

<font color='steelblue'>Indicate the actions that have been carried out in this process</font>

- Se han guardado los datos de turismo interior por provincia de origen 
- Se ha creado la variable "secreto" para distinguir si es necesario los datos faltantes

#### <font color=' #2874a6 '>Acctions to perform</font>

<font color='steelblue'>Indicate the actions that must be carried out in subsequent processes</font>

- Se deben unir a los datos de establecimientos por municipio para los modelos

## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay nada que desplegar en el piloto, ya que estos datos son estáticos o en todo caso cambian con muy poca frecuencia, altamente improbable durante el proyecto.


<font color=' red '><b>CODE</b></font>

In [42]:
# incluir código